

# Quickstart

Get started here. We will assess a payment default prediction model for performance and gender [fairness](https://www.credo.ai/glossary/fairness) using Lens, in 5 minutes. More in-depth information can be found in the [lens FAQ](https://credoai-lens.readthedocs.io/en/stable/notebooks/lens_faq.html#How-can-I-choose-which-assessments-to-run?)

**Setup**

Lens installation instruction can be found on [readthedocs](https://credoai-lens.readthedocs.io/en/stable/pages/setup.html)

**Find the code**

### Get your ML environment ready
 In this tutorial we will emulate the modeling phase by running a quick script. This script loads a dataset, splits it into training and testing, and fits a model. You can see the full script [here](https://raw.githubusercontent.com/credo-ai/credoai_lens/develop/docs/notebooks/training_script.py).

Here we have a gradient boosted classifier trained on the UCI Credit Card Default Dataset.

In [1]:
# model and X_test, y_test, etc. are defined by this script
from credoai.datasets import fetch_credit_model

# model and data are defined by this script
X_test, y_test, sensitive_features_test, model = fetch_credit_model()

2023-05-01 17:50:07,634 - lens - WARNING - 
            You are using credoai-lens version 1.1.8, however a newer version is available.
            Lens is updated regularly with major improvements and bug fixes.
            Please upgrade via the command: "python -m pip install --upgrade credoai-lens"
            


### Imports

In [2]:
# Import Lens and necessary artifacts
from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData

2023-05-01 17:50:12,812 - connect - WARNING - 
            You are using credoai-connect version 0.1.2, however a newer version is available.
            Lens is updated regularly with major improvements and bug fixes.
            Please upgrade via the command: "python -m pip install --upgrade credoai-connect"
            


/opt/homebrew/Caskroom/miniforge/base/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In lens, the classes that evaluate models and/or datasets are called `evaluators`. In this example we are interested in evaluating the model's fairness. For this we can use the `ModelFairness` evaluator. We'll
also evaluate the model's performance.

In [3]:
from credoai.evaluators import ModelFairness, Performance, ModelEquity, DataProfiler

## Lens in 5 minutes

Below is a basic example where our goal is to evaluate the above model. We'll break down this code [below](#Breaking-Down-The-Steps).

Briefly, the code is doing four things:

* Wrapping ML artifacts (like models and data) in Lens objects
* Initializing an instance of Lens. Lens is the main object that performs evaluations. Under the hood, it creates a `pipeline` of evaluations that are run.
* Add evaluators to Lens.
* Run Lens

In [4]:
# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)
credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

# Initialization of the Lens object
lens = Lens(model=credo_model, assessment_data=credo_data)

# initialize the evaluator and add it to Lens
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))
lens.add(ModelEquity())
lens.add(DataProfiler())


# run Lens
lens.run()

2023-05-01 17:50:14,667 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: SEX
2023-05-01 17:50:14,950 - lens - INFO - Evaluator Performance added to pipeline. 
2023-05-01 17:50:15,069 - lens - INFO - Evaluator ModelEquity added to pipeline. Sensitive feature: SEX
2023-05-01 17:50:15,071 - lens - INFO - Evaluator DataProfiler added to pipeline. 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s finished


### Getting results within your python environment

`lens.get_results()` provides a list where the results of the evaluators (a list of dataframes) are stored along with the evaluator metadata. In this case, there are 2 results - one for each evaluator.

In [5]:
results = lens.get_results()
print(f"Results for {len(results)} evaluators")

Results for 4 evaluators


`lens.get_results()` has some arguments, which makes it easier for you to get a subset of results. These are the same arguments that can be passed to `lens.get_pipeline` and `lens.get_evidence`.

In [6]:
performance_results = lens.get_results(evaluator_name='Performance')[0]
# first dataframe is overall metrics
display(performance_results['results'][0])
# second dataframe is the long form of the confusion matrix
display(performance_results['results'][1])

,type,value
0,precision_score,0.628081
1,recall_score,0.360172


,true_label,predicted_label,value
0,0,0,0.940916
1,1,0,0.639828
2,0,1,0.059084
3,1,1,0.360172


## Using Len's pipeline argument

If we want to add multiple evaluators to our pipeline, one way of doing it could be repeating the `add` step, as shown above. Another way is to define the pipeline steps, and pass it to `Lens` at initialization time. Let's explore the latter!

In [7]:
pipeline = [
    (ModelFairness(metrics)),
    (Performance(metrics)),
]
lens = Lens(model=credo_model, assessment_data=credo_data, pipeline=pipeline)

2023-05-01 17:50:15,649 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: SEX
2023-05-01 17:50:15,919 - lens - INFO - Evaluator Performance added to pipeline. 


Above, each of the `tuples` in the `list` is in the form `(instantiated_evaluator, id)`.

In [8]:
# notice that Lens functions can be chained together
results = lens.run().get_results()
print(f'\nFound results for {len(results)} evaluators')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished

Found results for 2 evaluators


Let's check that we have results for both of our evaluators. We'll print the metadata for each evaluation run followed by the dataframes for each result.

In [9]:
for r in results:
    print('\n'.join([f"{k}: {v}" for k, v in r['metadata'].items()]))
    for df in r['results']:
        display(df)
    print('\n')



evaluator: ModelFairness
sensitive_feature: SEX
dataset_type: assessment_data


,type,value
0,equal_opportunity,0.027686
0,precision_score_parity,0.016322
1,recall_score_parity,0.027686


,SEX,type,value
0,female,precision_score,0.618687
1,male,precision_score,0.635009
2,female,recall_score,0.344585
3,male,recall_score,0.372271


,true_label,predicted_label,value,sens_feat_group
0,0,0,0.935304,female
1,1,0,0.655415,female
2,0,1,0.064696,female
3,1,1,0.344585,female
4,0,0,0.944617,male
5,1,0,0.627729,male
6,0,1,0.055383,male
7,1,1,0.372271,male




evaluator: Performance


,type,value
0,precision_score,0.628081
1,recall_score,0.360172


,true_label,predicted_label,value
0,0,0,0.940916
1,1,0,0.639828
2,0,1,0.059084
3,1,1,0.360172


## That's it!

That should get you up and running. Next steps include:

* Trying out other evaluators (they are all accessible via `credoai.evaluators`)
* Checking out our developer guide to better understand the Lens ecosystem and see how you can extend it.
* Exploring the Credo AI Governance Platform, which will connect AI assessments with customizable governance to support reporting, compliance, multi-stakeholder translation and more!

## Breaking Down The Steps

### Preparing artifacts

Lens interacts with "AI Artifacts" which wrap model and data objects and standardize them for use by different evaluators.

Below we create a `ClassificationModel` artifact. This is a light wrapper for any kind of fitted classification model-like object. 

We also create a `TabularData` artifact which stores X, y and sensitive features.

In [10]:
# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)

credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

#### Model

Model type objects, like [ClassificationModel](https://credoai-lens.readthedocs.io/en/latest/_autosummary/credoai.artifacts.model.classification_model.html#module-credoai.artifacts.model.classification_model) used above, serve as adapters between arbitrary models and the evaluators in Lens. Some evaluators depend on Model instantiating certain methods. For example, `ClassificationModel` can accept any generic object having `predict` and `predict_proba` methods, including fitted sklearn pipelines.



#### Data

_Data_ type artifact, like `TabularData` serve as adapters between datasets and the evaluators in Lens.

When you pass data to a _Data_ artifact, the artifact performs various steps of validation, and formats them so that they can be used by evaluators. The aim of this procedure is to preempt errors down the line.

You can pass Data to Lens as a **training dataset** or an **assessment dataset** (see lens [class documentation](https://credoai-lens.readthedocs.io/en/latest/_autosummary/credoai.lens.lens.Lens.html#credoai.lens.lens.Lens)). If the former, it will not be used to assess the model. Instead, dataset assessments will be performed on the dataset (e.g., fairness assessment). The validation dataset will be assessed in the same way, but _also_ used to assess the model, if provided.

Similarly to _Model_ type objects, _Data_ objects can be customized, see [documentation](https://credoai-lens.readthedocs.io/en/latest/_autosummary/credoai.artifacts.data.html#module-credoai.artifacts.data).

### Evaluators 

Lens uses the above artifacts to ensure a successfull run of the evaluators. As we have seen in the sections [Lens in 5 minutes](#Lens-in-5-minutes) and [Adding multiple evaluators](#Using-Len's-pipeline-argument), multiple evaluators can be added to _Lens_ pipeline. Each evaluators contains information on what it needs in order to run successfully, and it executes a validation step at _add_ time.

The result of the validation depends on what artifacts are available, their content and the type of evaluator being added to the pipeline. In case the validation process fails, the user is notified the reason why the evaluator cannot be added to the pipeline.

See for example:

In [11]:
from credoai.evaluators import DataFairness
lens.add(DataFairness())

2023-05-01 17:50:16,067 - lens - INFO - Evaluator DataFairness added to pipeline. Sensitive feature: SEX


Currently no automatic run of evaluators is supported. However, when Lens is used in combination with Credo AI Platform, it is possible to download an assessment plan which then gets converted into a set of evaluations that Lens can run programmatically. For more information see the [governance tutorial](https://credoai-lens.readthedocs.io/en/stable/notebooks/platform_integration.html).

### Run Lens

After we have initialized _Lens_ the _Model_ and _Data_ (`ClassificationModel` and `TabularData` in our example) type artifacts, we can add whichever evaluators we want to the pipeline, and finally run it!

In [12]:
lens = Lens(model=credo_model, assessment_data=credo_data)
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.run()

2023-05-01 17:50:16,311 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: SEX
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


As you can notice, when adding _evaluators_ to lens, they need to be instantiated. If any extra arguments need to be passed to the evaluator (like metrics in this case), this is the time to do it.

**Getting Evaluator Results**

Afte the pipeline is run, the results become accessible via the method `get_results()`

`lens.get_results()` provides a dictionary where the results of the evaluators are stored as values, and the keys correspond to the ids of the evaluators.  

In the previous case we specified the id of the evaluator when we added `ModelFairness` to the pipeline, however `id` is an optional argument for the `add` method. If omitted, a random one will be generated.

In [13]:
lens.get_results()

[{'metadata': {'evaluator': 'ModelFairness',
   'sensitive_feature': 'SEX',
   'dataset_type': 'assessment_data'},
  'results': [                     type     value
   0       equal_opportunity  0.030618
   0  precision_score_parity  0.030256
   1     recall_score_parity  0.030618,
         SEX             type     value
   0  female  precision_score  0.267436
   1    male  precision_score  0.237179
   2  female     recall_score    0.7173
   3    male     recall_score  0.686681,
      true_label predicted_label     value sens_feat_group
   0           0               0  0.401457          female
   1           1               0  0.282700          female
   2           0               1  0.598543          female
   3           1               1  0.717300          female
   4           0               0  0.428370            male
   5           1               0  0.313319            male
   6           0               1  0.571630            male
   7           1               1  0.686681  

**Credo AI Governance Platform**

For information on how to interact with the plaform, please look into: [Connecting with Governance App](https://credoai-lens.readthedocs.io/en/stable/notebooks/platform_integration.html) tutorial for directions.
